In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ERROR: Error in library(tidyverse): there is no package called 'tidyverse'


In [2]:
library(tidyverse)
library(ggrepel)
library(reshape2)
library(ggExtra)
library(GGally)
library(hrbrthemes)

ERROR: Error in library(tidyverse): there is no package called 'tidyverse'


In [7]:
data <- read.csv("Medicalpremium.csv")

# **Exploratory Data Analysis**

In [8]:
head(data)

Age,Diabetes,BloodPressureProblems,AnyTransplants,AnyChronicDiseases,Height,Weight,KnownAllergies,HistoryOfCancerInFamily,NumberOfMajorSurgeries,PremiumPrice
45,0,0,0,0,155,57,0,0,0,25000
60,1,0,0,0,180,73,0,0,0,29000
36,1,1,0,0,158,59,0,0,1,23000
52,1,1,0,1,183,93,0,0,2,28000
38,0,0,0,1,166,88,0,0,1,23000
30,0,0,0,0,160,69,1,0,1,23000


In [9]:
str(data)

'data.frame':	986 obs. of  11 variables:
 $ Age                    : int  45 60 36 52 38 30 33 23 48 38 ...
 $ Diabetes               : int  0 1 1 1 0 0 0 0 1 0 ...
 $ BloodPressureProblems  : int  0 0 1 1 0 0 0 0 0 0 ...
 $ AnyTransplants         : int  0 0 0 0 0 0 0 0 0 0 ...
 $ AnyChronicDiseases     : int  0 0 0 1 1 0 0 0 0 0 ...
 $ Height                 : int  155 180 158 183 166 160 150 181 169 182 ...
 $ Weight                 : int  57 73 59 93 88 69 54 79 74 93 ...
 $ KnownAllergies         : int  0 0 0 0 0 1 0 1 1 0 ...
 $ HistoryOfCancerInFamily: int  0 0 0 0 0 0 0 0 0 0 ...
 $ NumberOfMajorSurgeries : int  0 0 1 2 1 1 0 0 0 0 ...
 $ PremiumPrice           : int  25000 29000 23000 28000 23000 23000 21000 15000 23000 23000 ...


In [10]:
dim(data)

[1] 986  11

# Correlation Matrix

In [11]:
ggcorr(data, label = T, color = "black", size = 5)+
  labs(title = "Correlation Matrix")+
  theme(plot.title = element_text(family = "Roboto Condensed", size = 19, face = "bold",vjust = 1),
        plot.subtitle = element_text(family = "Roboto Condensed", size = 16,vjust = 0))

ERROR: Error in ggcorr(data, label = T, color = "black", size = 5): could not find function "ggcorr"


##### It can be easily seen here that factor affecting Premium Price are Age, Number of Major Surgery, Any Transplant, Any Chronic Disease, Blood Pressure Problems, History of Cancer in Family, Diabeties, Weight. While factors that hardly matters are height and Known Allergies.
##### But it should be noted that correlation between factor might not neccessary mean a causation.

# **Applying factors to different colums**

In [12]:
data$Diabetes <- as.factor(data$Diabetes)
data$BloodPressureProblems <- as.factor(data$BloodPressureProblems)
data$AnyTransplants <- as.factor(data$AnyTransplants)
data$AnyChronicDiseases <- as.factor(data$AnyChronicDiseases)
data$KnownAllergies <- as.factor(data$KnownAllergies)
data$HistoryOfCancerInFamily <- as.factor(data$HistoryOfCancerInFamily)
data$NumberOfMajorSurgeries <- as.factor(data$NumberOfMajorSurgeries)


In [13]:
str(data)

'data.frame':	986 obs. of  11 variables:
 $ Age                    : int  45 60 36 52 38 30 33 23 48 38 ...
 $ Diabetes               : Factor w/ 2 levels "0","1": 1 2 2 2 1 1 1 1 2 1 ...
 $ BloodPressureProblems  : Factor w/ 2 levels "0","1": 1 1 2 2 1 1 1 1 1 1 ...
 $ AnyTransplants         : Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 1 1 ...
 $ AnyChronicDiseases     : Factor w/ 2 levels "0","1": 1 1 1 2 2 1 1 1 1 1 ...
 $ Height                 : int  155 180 158 183 166 160 150 181 169 182 ...
 $ Weight                 : int  57 73 59 93 88 69 54 79 74 93 ...
 $ KnownAllergies         : Factor w/ 2 levels "0","1": 1 1 1 1 1 2 1 2 2 1 ...
 $ HistoryOfCancerInFamily: Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 1 1 ...
 $ NumberOfMajorSurgeries : Factor w/ 4 levels "0","1","2","3": 1 1 2 3 2 2 1 1 1 1 ...
 $ PremiumPrice           : int  25000 29000 23000 28000 23000 23000 21000 15000 23000 23000 ...


# **CALCULATING BMI**

In [14]:
data$bmi <- 10000*(data$Weight/(data$Height)^2)

# **ASSIGNING CATEGORIES TO DIFFERENT BMI RANGES**

In [15]:
data <- data %>%
  mutate( bmiCategory = case_when(
                                  bmi<18.49999 ~ "under weight",
                                  bmi>18.5 & bmi<24.99999 ~ "normal weight",
                                  bmi>25 & bmi<29.99999 ~ "over weight",
                                  bmi>30 ~ "obesity"
  ))

ERROR: Error in data %>% mutate(bmiCategory = case_when(bmi < 18.49999 ~ "under weight", : could not find function "%>%"


#### Note that 0 means the absence of an attribute while 1 shows it's presence

#  Diabetics People Premium Analysis

### Average Difference in Premium Prices for Diabetic and Non-Diabetic People

In [16]:
data %>% 
  select(Diabetes,PremiumPrice) %>%
  group_by(Diabetes) %>% 
  summarise( PremiumPrice = mean(PremiumPrice)) %>% 
  ggplot(.,aes(Diabetes,PremiumPrice))+
  geom_bar(stat = "identity",width = 0.4, fill = "#56B4E9", alpha = 0.6)+
  labs(title = "Bar plot for Diabetics People")

ERROR: Error in data %>% select(Diabetes, PremiumPrice) %>% group_by(Diabetes) %>% : could not find function "%>%"


### Distribution of Premium Prices for Diabetic and Non-Diabetic Peoples

In [17]:
ggplot(data, aes(PremiumPrice))+
  geom_density(aes(fill = Diabetes), color = NA, alpha = 0.6)+
  labs(title = "Density plot for Diabetics and Non-diabetic proples")

ERROR: Error in ggplot(data, aes(PremiumPrice)): could not find function "ggplot"


#  Blood Pressure Patients Premium Analysis

### Average Difference in Premium Prices for Blood Pressure patients and Non-Blood Pressure Patients

In [18]:
data %>% 
  select(BloodPressureProblems,PremiumPrice) %>%
  group_by(BloodPressureProblems) %>% 
  summarise( PremiumPrice = mean(PremiumPrice)) %>% 
  ggplot(.,aes(BloodPressureProblems,PremiumPrice))+
  geom_bar(stat = "identity",width = 0.4, fill = "#56B4E9", alpha = 0.6)+
  labs(title = "Bar plot for blood pressure problem")

ERROR: Error in data %>% select(BloodPressureProblems, PremiumPrice) %>% group_by(BloodPressureProblems) %>% : could not find function "%>%"


### Distribution of Premium Prices for Blood Pressure Patients and Non-Blood Pressure Patients

In [19]:
ggplot(data, aes(PremiumPrice))+
  geom_density(aes(fill = BloodPressureProblems), color = NA, alpha = 0.6)+
  labs(title = "Density plot for people with and without blood pressure problems")

ERROR: Error in ggplot(data, aes(PremiumPrice)): could not find function "ggplot"


#  People Gone Through Any Transplants Premium Analysis

### Average Difference in Premium Prices for People Gone Through Any Transplants vs Those Who haven't Gone Through Any Transplants

In [20]:
data %>% 
  select(AnyTransplants,PremiumPrice) %>%
  group_by(AnyTransplants) %>% 
  summarise( PremiumPrice = mean(PremiumPrice)) %>% 
  ggplot(.,aes(AnyTransplants,PremiumPrice))+
  geom_bar(stat = "identity",width = 0.4, fill = "#56B4E9", alpha = 0.6)+
  labs(title = "Bar plot for people gone through any transplants")

ERROR: Error in data %>% select(AnyTransplants, PremiumPrice) %>% group_by(AnyTransplants) %>% : could not find function "%>%"


### Distribution of Premium Prices for People Gone Through Any Transplants vs Those Who haven't Gone Through Any Transplants

In [21]:
ggplot(data, aes(PremiumPrice))+
  geom_density(aes(fill = AnyTransplants), color = NA, alpha = 0.6)+
  labs(title = "Density plot for people gone through any transpalnts")


ERROR: Error in ggplot(data, aes(PremiumPrice)): could not find function "ggplot"


#  People With Chronic Disease Premium Analysis

### Average Difference in Premium Prices for People With Chronic Disease  and People With No Chronic Disease 

In [22]:
data %>% 
  select(AnyChronicDiseases,PremiumPrice) %>%
  group_by(AnyChronicDiseases) %>% 
  summarise( PremiumPrice = mean(PremiumPrice)) %>% 
  ggplot(.,aes(AnyChronicDiseases,PremiumPrice))+
  geom_bar(stat = "identity",width = 0.4, fill = "#56B4E9", alpha = 0.6)+
  labs(title = "Bar plot for people with cronic disease")

ERROR: Error in data %>% select(AnyChronicDiseases, PremiumPrice) %>% group_by(AnyChronicDiseases) %>% : could not find function "%>%"


### Distribution of Premium Prices for People With Chronic Disease and People With No Chronic Disease 

In [23]:
ggplot(data, aes(PremiumPrice))+
  geom_density(aes(fill = AnyChronicDiseases), color = NA, alpha = 0.6)+
  labs(title = "Density plot for  having chronic diseases")


ERROR: Error in ggplot(data, aes(PremiumPrice)): could not find function "ggplot"


#  Allergy Patients Premium Analysis

### Average Difference in Premium Prices for Allergy patients and No Allergy Patients

In [24]:
data %>% 
  select(KnownAllergies,PremiumPrice) %>%
  group_by(KnownAllergies) %>% 
  summarise( PremiumPrice = mean(PremiumPrice)) %>% 
  ggplot(.,aes(KnownAllergies,PremiumPrice))+
  geom_bar(stat = "identity",width = 0.4, fill = "#56B4E9", alpha = 0.6)+
  labs(title = "Bar plot for people having allergies")

ERROR: Error in data %>% select(KnownAllergies, PremiumPrice) %>% group_by(KnownAllergies) %>% : could not find function "%>%"


### Distribution of Premium Prices for Allergy Patients and No Allergy Patients

In [25]:
ggplot(data, aes(PremiumPrice))+
  geom_density(aes(fill = KnownAllergies), color = NA, alpha = 0.6)+
  labs(title = "Density plot for people with and without allergies")


ERROR: Error in ggplot(data, aes(PremiumPrice)): could not find function "ggplot"


#  Patients with History of Cancer in Family Premium Analysis

### Average Difference in Premium Prices for Patients with History of Cancer and Patients without History of Cancer

In [26]:
data %>% 
  select(HistoryOfCancerInFamily,PremiumPrice) %>%
  group_by(HistoryOfCancerInFamily) %>% 
  summarise( PremiumPrice = mean(PremiumPrice)) %>% 
  ggplot(.,aes(HistoryOfCancerInFamily,PremiumPrice))+
  geom_bar(stat = "identity",width = 0.4, fill = "#56B4E9", alpha = 0.6)+
  labs(title = "Bar plot for people with history of cancers")

ERROR: Error in data %>% select(HistoryOfCancerInFamily, PremiumPrice) %>% group_by(HistoryOfCancerInFamily) %>% : could not find function "%>%"


### Distribution of Premium Prices for Patients with History of Cancer and Patients without History of Cancer

In [27]:
ggplot(data, aes(PremiumPrice))+
  geom_density(aes(fill = HistoryOfCancerInFamily), color = NA, alpha = 0.6)+
  labs(title = "Density plot for people with and without history of cancers")



ERROR: Error in ggplot(data, aes(PremiumPrice)): could not find function "ggplot"


#  People Gone Through Major Surgeries Premium Analysis

### Average Difference in Premium Prices for People gone through major surgeries

In [28]:
data %>% 
  select(NumberOfMajorSurgeries,PremiumPrice) %>%
  group_by(NumberOfMajorSurgeries) %>% 
  summarise( PremiumPrice = mean(PremiumPrice)) %>% 
  ggplot(.,aes(NumberOfMajorSurgeries,PremiumPrice))+
  geom_bar(stat = "identity",width = 0.4, fill = "#56B4E9", alpha = 0.6)+
  labs(title = "Bar plot for major surgeries")

ERROR: Error in data %>% select(NumberOfMajorSurgeries, PremiumPrice) %>% group_by(NumberOfMajorSurgeries) %>% : could not find function "%>%"


### Distribution of Premium Prices for People gone through major surgeries
#####  Here people with 4 are charged constant 28000 hence neglected that uniform distribution

In [29]:
ggplot(data %>% 
         select(NumberOfMajorSurgeries,PremiumPrice) %>%
         filter(!NumberOfMajorSurgeries == 3),
         aes(PremiumPrice))+
  geom_density(aes(fill = NumberOfMajorSurgeries), color = NA, alpha = 0.6)+
  labs(title = "Density plot for people with differnt number of surgeries")


ERROR: Error in ggplot(data %>% select(NumberOfMajorSurgeries, PremiumPrice) %>% : could not find function "ggplot"


# Box and Whisker Plot for Different BMI category 

In [30]:
data %>% 
  mutate(bmiCategory = str_to_title(bmiCategory)) %>% 
  
  ggplot(aes(bmiCategory, PremiumPrice))+
  geom_boxplot()+
  geom_jitter(aes(color = bmiCategory),alpha = 0.4)+
  labs(title = "Distribution of Premium Price Over Differnt BMI category")+
  theme_ipsum_rc()

ERROR: Error in data %>% mutate(bmiCategory = str_to_title(bmiCategory)) %>% : could not find function "%>%"


# **Prediction Model**

In [31]:
library(randomForest)
library(caTools)

ERROR: Error in library(randomForest): there is no package called 'randomForest'


In [32]:
data$PremiumPrice <- as.factor(data$PremiumPrice)
summary(data)

      Age        Diabetes BloodPressureProblems AnyTransplants
 Min.   :18.00   0:572    0:524                 0:931         
 1st Qu.:30.00   1:414    1:462                 1: 55         
 Median :42.00                                                
 Mean   :41.75                                                
 3rd Qu.:53.00                                                
 Max.   :66.00                                                
                                                              
 AnyChronicDiseases     Height          Weight       KnownAllergies
 0:808              Min.   :145.0   Min.   : 51.00   0:774         
 1:178              1st Qu.:161.0   1st Qu.: 67.00   1:212         
                    Median :168.0   Median : 75.00                 
                    Mean   :168.2   Mean   : 76.95                 
                    3rd Qu.:176.0   3rd Qu.: 87.00                 
                    Max.   :188.0   Max.   :132.00                 
                    

### **Spliting Data into two Subset for Training and Testing use.**

In [33]:
sample <- sample.split(data$PremiumPrice, SplitRatio = 0.75)
train <- subset(data, sample == TRUE)
test <- subset(data, sample == FALSE)

ERROR: Error in sample.split(data$PremiumPrice, SplitRatio = 0.75): could not find function "sample.split"


In [34]:
dim(train)
dim(test)

ERROR: Error in eval(expr, envir, enclos): object 'train' not found


#### Creating Random Forest Model

In [35]:
rf <- randomForest(
  PremiumPrice ~ .,
  data=train
)

ERROR: Error in randomForest(PremiumPrice ~ ., data = train): could not find function "randomForest"


In [36]:
print(rf)

function (n, df1, df2, ncp) 
{
    if (missing(ncp)) 
        .Call(C_rf, n, df1, df2)
    else (rchisq(n, df1, ncp = ncp)/df1)/(rchisq(n, df2)/df2)
}
<bytecode: 0x000000003f7374f8>
<environment: namespace:stats>


In [37]:
pred <- predict(rf, newdata = test[-11])

ERROR: Error in UseMethod("predict"): no applicable method for 'predict' applied to an object of class "function"


In [38]:
cm <- table(pred,obs = test[,11])


ERROR: Error in table(pred, obs = test[, 11]): object 'pred' not found


In [39]:
sum <- 0
for (i in 1:24){
    for(j in 1:24){
        if(i!=j){
            sum <-sum+cm[i,j]
        }
    }
}
sum
print(paste("The Accuracy of Random Forest Model is",(243-sum)/2.43))

ERROR: Error in cm[i, j]: object of type 'closure' is not subsettable


#### **The prediction when using 75% data for training and 25% for testing turnout to be 93% using Random Forest technique.**